# Demo Lesson: Helicopter Drop

In [ ]:
# Run this cell to set up the notebook, but please don't change it.

# These lines import the Numpy and Datascience modules.
import numpy as np
from datascience import *

# These lines do some fancy plotting
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

## 1. A/B Testing

A/B testing is a form of hypothesis testing that allows you to make comparisons between two distributions.

You'll almost never be explicitly asked to perform an A/B test. Make sure you can identify situations where the test is appropriate and know how to correctly implement each step.

### Question 1.1.

The following statements are the unordered steps of an A/B hypothesis test:

1. Choose a test statistic (typically the difference in means between two categories)

2. Shuffle the labels of the original sample, find your simulated test statistic, and repeat many times

3. Find the value of the observed test statistic

4. Calculate the p-value based off your observed and simulated test statistics

5. Define a null and alternate model

6. Use the p-value and p-value cutoff to draw a conclusion about the null hypothesis

Make an array called `ab_test_order` that contains the correct order of an A/B test, where the first item of the array is the first step of an A/B test and the last item of the array is the last step of an A/B test

In [ ]:
ab_test_order = make_array(5, 1, 3, 2, 4, 6) # SOLUTION

### Question 1.2

If the null hypothesis of an A/B test is correct, should the order of labels affect the differences in means between each group?

**SOLUTION:** Under the null model, there should be no statistically significant difference between the grouped means.

### Question 1.3

Why do we shuffle labels in an A/B test? 

**SOLUTION:** We shuffle labels as a way to resample the original data, and understand the baseline differences between groups in the data.

## 2. Paper Helicopters

The Paper Helicopter Experiment (https://www.paperhelicopterexperiment.com/) provides different templates for constructing paper helicopters that can be dropped. The designs can be modified to create helicopters with different dimensions for the wings, body, weighting, and more. In this lab you'll investigate whether the length of the rotor and the amount of weight attached to helicopter results in a different in the time it takes to fall from a 3rd floor stairwell. Do conduct your analysis, you'll use an A/B test.

### The Data
The data was collected by a group of NCSSM Online students during online weekend held October 2nd, 2021. Five groups of students dropped 143 paper helicopters in total. There were 4 unique configurations that were assigned to the groups, one per group.

1. The Default: Full length (long) rotors, unweighted body
2. The Short: Short rotors, unweighted body
3. The Heavy: Long rotors, weighted body
4. The Short Heavy: Short rotors, weighted body

The following features were recorded for each helicopter dropped:
* `Rotor`: Long or Short (string). Indicates if the helicopter had full length or shortened rotors.
* `Weight`: Weighted or Unweighted (string). Indicates if the helicopter had a weighted (3 paperclips) or unweighted body.
* `Team`: 1, 2, 3, 4, or 5 (int). Indicates which team
* `Location`: Side A or Side B (string). Indicates which side of the building the helicopters were dropped from.
* `Obstruction`: True or False (Boolean). Indicates if the helicopter hit the stairwell or other object on the way down.
* `Time`: (Float). Time, measured in seconds, it took for the helicopter to hit the ground after being released.

Run the following cell to load the data in as the table named `copters`.

In [ ]:
copters = Table.read_table('helicopters.csv')
copters

Due to variation in the construction, weather, and randomness of the universe not every drop for each configuration was identical. Let's explore the data a bit to see how the drop times varied.

### Question 2.1.

Let's start by cleaning the data a little. This dataset has several measurements taken where the helicopter hit the stairwell, a nearby building, or some other obstruction that impacted the fall time. These results should be discarded since they wouldn't be an accurate reflection of what we're trying to measure.

Create a new table called `helicopters` that only contains the rows of `copters` that correspond to a helicopter that did not hit an obstruction during the fall. Additionally, to focus our initial investigation only keep the data on those helicopters that were unweighted.

In [ ]:
helicopters = copters.where('Obstruction', False).where('Weight', 'Unweighted') # SOLUTION
helicopters

### Question 2.2
Suppose you're interested in determining how the rotor length might impact fall time. Start by creating a histogram that displays the distribution of times for helicopters with long rotors. Use the provided bins stored to `my_bins`.

In [ ]:
my_bins = np.arange(2, 9.5, 0.5)
helicopters.where('Rotor', 'Long').hist('Time', bins=my_bins ) # SOLUTION

### Question 2.3

Now, construct a histogram that shows the distribution of times for only helicopters with short rotors. Again, use the provided bins.

In [ ]:
my_bins = np.arange(2, 9.5, 0.5)
helicopters.where('Rotor', 'Short').hist('Time', bins=my_bins ) # SOLUTION

### Question 2.4

It can be difficult sometimes to compare two histograms when they are on separate graphs. However, by providing the argument `group` to the `.hist()` method with the label (string) for a column you wish to group by, and it will produce a histogram for each unique group.

For example, `table.hist('col_label', bins=your_bins, group='feature')` would produce a histogram for each unique group found in the `feature` column of `table`, using the numerical data in the column labeled `col` label.

Running the cell below will create 2 histograms, one for each group as determined by the values in the `Rotor` column.

In [ ]:
my_bins = np.arange(2, 9.5, 0.5)
helicopters.hist('Time', bins=my_bins, group="Rotor" )

You can see it's much easier to compare these two distributions now that they're on the same scale and set of axes.

## 3. The Rotor Question

You might now be wondering if there is a difference between the fall times of helicopters with long (full length) vs. short rotors. It certainly appears that the distributions might be different from one another. We'll use our data to attempt to draw a conclusion using simulation.

### Question 3.1.

We want to know whether the shortening the rotor length **causes** a change in the fall time.  

Why is it not sufficient to compare fall times of the helicopters with short rotors and those with long rotors?

**SOLUTION:** Only comparing fall times to each other is not a randomized control experiment and would leave our conclusion open to influence by confounding factors. Additionally, we wouldn't be able to determine if any differences we observed are typical due to chance variations alone.

### Question 3.2.

Why is an A/B test appropriate for this situation? What would you use as the "A" group (the control group) and which group is the "B" group (the treatment group)?

**SOLUTION:** An A/B test is appropriate because we are comparing 2 distributions of numerical data. The "A" group is full length rotors. The "B" group is the short rotors. 

### Question 3.3.

Create a table `time_means` that contains the average fall time for the full length rotor helicopters and the shortened rotor helicopters. It should have two columns: one indicating the rotor length, and one that contains the average fall time for each group.

**Hint:** Use a combination of `.select()` and `.group()` to make short work of this.

```
BEGIN QUESTION
name: q3_3
```

In [ ]:
time_means = helicopters.select("Rotor", "Time").group('Rotor', np.average) # SOLUTION
time_means

### Question 3.4

We want to figure out if there is a difference between the distribution of fall times for full length and shortened rotor helicopters. Specifically, we want to test if fall times were longer for longer rotor helicopters than they were for shorter rotor helicopters. 

Define the following for this analysis:
* Null hypothesis
* Alternative hypothesis
* Test statistic

How does the test statistic help us differentiate whether the data supports the null and alternative? 

Confirm your answer with your instructor (or a classmate who already has) before moving on.

**SOLUTION:** We want to find the difference between the mean fall times. Large negative values of this difference support the alternative hypothesis. Small values support the null.

### Question 3.5.

Calculate the test statistic for our observed data using the `time_means` table. Set `observed_difference` to the value you compute. 

```
BEGIN QUESTION
name: q3_5
```

In [ ]:
means = time_means.column(1)
observed_difference = means.item(1) - means.item(0)
observed_difference

### Question 3.6

To perform an A/B test, we'll need to compute similar statistics for our many simulations we'll run. Writing a function which returns the statistic will be a great way to save time. Write a function named `find_test_stat` which takes in the arguments `table`, `labels_col`, and `values_col` that calculates the test statistic for this problem. The `table` passed into this function will be a permutation of our original table and structured the same way. `labels_col` will be passed a string that specifies which column in the table contains the label you'll be grouping by. `values_col` specifies which column contains the values that you'll be using to compute the test statistic.

When you've written this function, you can pass it any table and the two specified labels and it will compute a test statistic for an A/B test, not just for this problem, but any problem! A very handy function to keep around. 

Running `find_test_stat(helicopters, "Rotor", "Time")` should return the exact same value your code from the previous question computed: the difference between of average values in the "Time column between the groups determined by "Rotor" column in the `helicopters` table. However, you should also be able to run `find_test_stat(helicopters, "Weight", "Time")` to compute the difference between of average values in the **"Weight"** column between the groups determined by "Rotor" column in the `helicopters` table just as easily. 

```
BEGIN QUESTION
name: q3_6
```

In [ ]:
def find_test_stat(table, labels_col, values_col):
    # BEGIN SOLUTION
    reduced = table.select(labels_col, values_col)
    means_table = reduced.group(labels_col, np.average)
    means = means_table.column(1)
    return means.item(1) - means.item(0)
    # END SOLUTION
    
find_test_stat(helicopters, "Rotor", "Time")

When we run a simulation for A/B testing, we resample the data by shuffling the labels of the original sample. If the null hypothesis is true and the time distributions **are** the same, we expect that the difference in mean time will be not change when "Rotor" labels are changed.

### Question 3.7.

Write a function `simulate_and_test_statistic` to compute one trial of our A/B test. Your function should run a simulation and return a test statistic.

**Hint:** You can "shuffle" the labels by using `.sample(with_replacement = False)` on the table, and then just selected the column (as an array using `.column`) that contains the newly shuffled labels. Then, you can either overwrite the existing labels, or, extend the table with a new column labeled something like "shuffled labels". Just make sure you pass the correct label on to `find_test_stat`!

In [ ]:
def simulate_and_test_statistic(table, labels_col, values_col):
    shuffled_labels = table.sample(with_replacement = False).column(labels_col)
    original_and_shuffled = table.with_column("shuffled labels", shuffled_labels)
    return find_test_stat(original_and_shuffled, "shuffled labels", values_col)

simulate_and_test_statistic(helicopters, "Rotor", "Time")

### Question 3.8

Now that we can quickly shuffle the labels and compute a test statistic, simulate 5000 trials of our A/B test and store the test statistics in an array called `differences`

In [ ]:
# This cell might take a couple seconds to run

differences = make_array()
repetitions = 5000

for i in np.arange(repetitions):
    new_difference = simulate_and_test_statistic(helicopters, "Rotor", "Time")
    differences = np.append(differences, new_difference)

differences

Run the cell below to view a histogram of your simulated test statistics plotted with your observed test statistic.

In [ ]:
Table().with_column('Difference Between Group Means', differences).hist()
plt.scatter(observed_difference, -0.01, color = 'red', s = 60, zorder = 2);

### Question 3.9.

Find the p-value for your test and assign it to `empirical_P`

In [ ]:
empirical_P = np.count_nonzero(differences <= observed_difference) / repetitions
empirical_P

### Question 3.10.

Using a 1% P-value cutoff, draw a conclusion about the null and alternative hypotheses. Describe your findings using simple, non-technical language. What does your analysis tell you about fall times of helicopters with full length vs shortened rotors?

**SOLUTION:** Our empirical p-value (I computed a value of 0.0394) is less than than the cutoff that was set at 0.01. The data from our simulation would support our alternative hypothesis, that rotor length

## 4. Additional Investigation

Suppose you're curious if the stairwell that similar helicopters were dropped from would cause a difference in drop times. Let's focus on one type of helicopter, the full size rotor and unweighted body, which was dropped from both stairwells. Run the code below to create a table called `stairwell` which will pare down the original data set to just this type of helicopter.

In [ ]:
stairwell = copters.where('Obstruction', False).where('Rotor', 'Long').where('Weight', 'Unweighted')
stairwell

Complete a similar analysis to:

* compute the observed value of the test statistic
* simulate 5000 A/B tests to create a distribution of the test statistic
* compute the empirical p-value
* determine if the stairwell caused a difference in the drop times
